In [3]:
pip install selenium webdriver-manager pandas


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [45]:
#TUTTE SCARPE (4 MARCHI)
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

# INPUT del colore 
colore = input("📄 Inserisci il colore delle scarpe (es. nero, rosso, blu): ").strip().lower()
# Associa ogni colore all'URL di ricerca corrispondente
color_urls = {
    "nero":  "https://www.amazon.it/s?k=scarpe&i=fashion&rh=n%3A5512286031%2Cp_123%3A193963%257C389207%257C597269%257C9437%2Cp_n_feature_thirty-three_browse-bin%3A96332046031%2Cp_n_size_two_browse-vebin%3A14223338031&dc&__mk_it_IT=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=1BZC206652037&qid=1747852639&rnid=14223337031&sprefix=scarpe%2Cfashion%2C353&ref=sr_nr_p_n_size_two_browse-vebin_14&ds=v1%3A1PzeTi1BLCu5KVubYGrFumIAM%2BpKk%2F8gks7h2Ia8WN4",
    "rosso": "https://www.amazon.it/s?k=scarpe&i=fashion&rh=n%3A5512286031%2Cp_123%3A193963%257C389207%257C597269%257C9437%2Cp_n_feature_thirty-three_browse-bin%3A96332046031%2Cp_n_size_two_browse-vebin%3A14223343031&dc&__mk_it_IT=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=1BZC206652037&qid=1747854369&rnid=14223337031&sprefix=scarpe%2Cfashion%2C353&ref=sr_nr_p_n_size_two_browse-vebin_11&ds=v1%3AqOsRINYmz5WAeK%2Fyoe9E2J%2B6pEHOWGPj2yjP5n9ZHYY",
    "marrone":   "https://www.amazon.it/s?k=scarpe&i=fashion&rh=n%3A5512286031%2Cp_123%3A193963%257C389207%257C597269%257C9437%2Cp_n_feature_thirty-three_browse-bin%3A96332046031%2Cp_n_size_two_browse-vebin%3A14223341031&dc&__mk_it_IT=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=1BZC206652037&qid=1747854411&rnid=14223337031&sprefix=scarpe%2Cfashion%2C353&ref=sr_nr_p_n_size_two_browse-vebin_14&ds=v1%3ApS3xEGdxgIxIZi9I4zW7O%2BuKRhD30cm21vF%2BU7%2Fq7A8",
    "beige": "https://www.amazon.it/s?k=scarpe&i=fashion&rh=n%3A5512286031%2Cp_123%3A193963%257C389207%257C597269%257C9437%2Cp_n_feature_thirty-three_browse-bin%3A96332046031%2Cp_n_size_two_browse-vebin%3A14223342031&dc&__mk_it_IT=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=1BZC206652037&qid=1747854471&rnid=14223337031&sprefix=scarpe%2Cfashion%2C353&ref=sr_nr_p_n_size_two_browse-vebin_9&ds=v1%3A2WqYxGi%2FZ4%2BrCWdr3wLYgiFe4dsE%2FtfnL3DLmq578WQ"
    # aggiungi qui gli altri colori e i loro URL
}
if colore not in color_urls:
    print(f"⚠️ Colore non riconosciuto. Scegli tra: {', '.join(color_urls.keys())}")
    exit(1)

start_url = color_urls[colore]
print(f"🔍 Avvio scraping per colore “{colore}” con URL:\n{start_url}\n")


#1) Setup Selenium con opzioni anti‐bot 
options = Options()
# commenta headless per minimizzare il fingerprinting
# options.headless = True
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option("useAutomationExtension", False)
options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/118.0.0.0 Safari/537.36"
)

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)


# 2) URL di partenza 
#start_url = (
    
    #"https://www.amazon.it/s?k=scarpe+stivaletti+donna&i=fashion&rh=n%3A5512286031%2Cp_123%3A193963%257C237326%257C389207%257C9437&dc=&__mk_it_IT=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=1KW1VKWJQPEUX&qid=1747845895&rnid=2492969031&sprefix=scarpe+stivaletti+do%2Cfashion%2C189&xpid=H08luOnxQvkrq&ref=sr_nr_p_36_0_0&low-price=290&high-price="
#)
driver.get(start_url)
time.sleep(7)

# 3) Loop di paginazione e scraping 
page = 1
all_brands = []
all_names = []
all_prices = []
all_ratings = []

while True:
    print(f"Scraping pagina {page}…")
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # Estrae i dati
    brand_elems   = soup.find_all('span', class_="a-size-base-plus a-color-base")
    name_elems    = soup.find_all("h2", class_="a-size-base-plus a-spacing-none a-color-base a-text-normal")
    price_elems   = soup.find_all("span", class_="a-offscreen")
    rating_elems  = soup.select("span.a-icon-alt")
   

    # Pulisce  i testi
    brands  = [b.text.strip() for b in brand_elems]
    names   = [n.text.strip() for n in name_elems]
    prices  = [p.text.strip() for p in price_elems]
    ratings = [r.text.strip() for r in rating_elems]

    #Pad per i nulli
    max_len = max(len(brands), len(names), len(prices), len(ratings))

    def pad_list(lst):
        return lst + [np.nan] * (max_len - len(lst))

    brands = pad_list(brands)
    names = pad_list(names)
    prices = pad_list(prices)
    ratings = pad_list(ratings)
    
        # Accumula
    all_brands  .extend(brands)
    all_names   .extend(names)
    all_prices  .extend(prices)
    all_ratings .extend(ratings)

    # Prova a cliccare il link della pagina successiva
    try:
        selector = f"a.s-pagination-item[aria-label='Vai alla pagina {page+1}']"
        next_btn = driver.find_element(By.CSS_SELECTOR, selector)

        # Porta il bottone al centro della viewport
        driver.execute_script(
            "arguments[0].scrollIntoView({block: 'center', inline: 'center'});",
            next_btn
        )
        time.sleep(0.5)

        # Click via JS per evitare intercettazioni
        driver.execute_script("arguments[0].click();", next_btn)

        page += 1
        time.sleep(7)  # attendi il caricamento della nuova pagina
    except NoSuchElementException:
        print("✅ Fine paginazione.")
        break

driver.quit()

# 4) Creazione DataFrame ed export CSV---
df = pd.DataFrame({
    "Marchio":     all_brands,
    "Nome":        all_names,
    "Prezzo":      all_prices,
    "Valutazione": all_ratings
})

output_file = f"amazon_4marche_{colore}.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")
print(f"[OK] CSV creato con {len(df)} righe: {output_file}")
print(df.head())



📄 Inserisci il colore delle scarpe (es. nero, rosso, blu):  beige


🔍 Avvio scraping per colore “beige” con URL:
https://www.amazon.it/s?k=scarpe&i=fashion&rh=n%3A5512286031%2Cp_123%3A193963%257C389207%257C597269%257C9437%2Cp_n_feature_thirty-three_browse-bin%3A96332046031%2Cp_n_size_two_browse-vebin%3A14223342031&dc&__mk_it_IT=%C3%85M%C3%85%C5%BD%C3%95%C3%91&crid=1BZC206652037&qid=1747854471&rnid=14223337031&sprefix=scarpe%2Cfashion%2C353&ref=sr_nr_p_n_size_two_browse-vebin_9&ds=v1%3A2WqYxGi%2FZ4%2BrCWdr3wLYgiFe4dsE%2FtfnL3DLmq578WQ

🔍 Scraping pagina 1…
🔍 Scraping pagina 2…
🔍 Scraping pagina 3…
🔍 Scraping pagina 4…
🔍 Scraping pagina 5…
🔍 Scraping pagina 6…
🔍 Scraping pagina 7…
🔍 Scraping pagina 8…
🔍 Scraping pagina 9…
🔍 Scraping pagina 10…
🔍 Scraping pagina 11…
🔍 Scraping pagina 12…
🔍 Scraping pagina 13…
🔍 Scraping pagina 14…
🔍 Scraping pagina 15…
🔍 Scraping pagina 16…
🔍 Scraping pagina 17…
🔍 Scraping pagina 18…
🔍 Scraping pagina 19…
🔍 Scraping pagina 20…
🔍 Scraping pagina 21…
✅ Fine paginazione.
[OK] CSV creato con 1169 righe: amazon_4marche_beige.c